# Vaccine Efficacy

Source : https://sachin-date.medium.com/how-to-estimate-vaccine-efficacy-using-a-logistic-regression-model-121f9ca5a9d8

$Vaccine Efficacy (VE)  = 1 - Incident Rate Ratio (IRR)$  
$IRR = \frac{p_{vacinated}} {p_{placebo}}$

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [6]:
# df = pd.read_csv('https://gist.githubusercontent.com/sachinsdate/2a6dc052b3c416e24a4d04c468358c44/raw/376939c4a1e8205f6bda03d0415b9758986ff693/vaccine_trial_simulation_study.csv')
df = pd.read_csv('vaccine_trial_simulation_study.csv')

In [7]:
df.head()

,INTERVAL_BETWEEN_DOSES,VACCINATED,INFECTED
0,0,1,1
1,0,1,1
2,0,1,1
3,0,1,1
4,0,1,1


### Analysis
1 Standard inference with statistical test  
2 Standard inference with Statsmodel  
3 Bootstraping inference  

### Basic Summary

In [9]:
pd.crosstab(index=df['VACCINATED'], columns=df['INFECTED'])

INFECTED,0,1
VACCINATED,,
0,5523,555
1,5891,31


In [10]:
pd.crosstab(index=df['VACCINATED'], columns=df['INFECTED'], margins=True)

INFECTED,0,1,All
VACCINATED,,,
0,5523,555,6078
1,5891,31,5922
All,11414,586,12000


In [11]:
pd.crosstab(index=df['VACCINATED'], columns=df['INFECTED'], margins=True, normalize=True)

INFECTED,0,1,All
VACCINATED,,,
0,0.460250,0.046250,0.5065
1,0.490917,0.002583,0.4935
All,0.951167,0.048833,1.0000


### 1) Test if the `INFECTED` independence of `VACCINATED`  

#### 1.1) $\chi^2$ test of independence, library `Scipy`  
reference : https://www.jmp.com/en_us/statistics-knowledge-portal/chi-square-test.html  
reference : https://stats.stackexchange.com/questions/110718/chi-squared-test-with-scipy-whats-the-difference-between-chi2-contingency-and  

In [22]:
from scipy.stats import chi2_contingency

In [23]:
contingency_tbl = pd.crosstab(index=df['VACCINATED'], columns=df['INFECTED'])

In [26]:
chi, p, ddof, expected = chi2_contingency(contingency_tbl)
expected

array([[5781.191,  296.809],
       [5632.809,  289.191]])

In [27]:
print(p)

1.1586205594709897e-105


With p = 1.15 e-105, chance of repeate testing and find the same result : dependence between 2 categorical variables

#### 1.2) $\chi^2$ test of independence, library `statsmodels.`
reference : https://www.statsmodels.org/stable/contingency_tables.html

In [16]:
contingency_tbl = pd.crosstab(index=df['VACCINATED'], columns=df['INFECTED'])
# Convert numpy table to Statsmodel contingency table
cont_tbl_sm = sm.stats.Table(contingency_tbl)

In [19]:
# original contingency table
cont_tbl_sm.table_orig

INFECTED,0,1
VACCINATED,,
0,5523,555
1,5891,31


In [28]:
# Expected contingency table if factor are independence
cont_tbl_sm.fittedvalues

INFECTED,0,1
VACCINATED,,
0,5781.191,296.809
1,5632.809,289.191


In [37]:
test_chi_indep = cont_tbl_sm.test_nominal_association()
test_chi_indep.pvalue

0.0

### 2) Test if proportion of `INFECTED` differ between `VACCINATED` group

#### 2.1) Sampling distribution test with $Z-test$ , library `scipy`

reference : https://online.stat.psu.edu/stat415/lesson/9/9.4

reference : https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.test_proportions_2indep.html#statsmodels.stats.proportion.test_proportions_2indep

`Assumption CTL` difference of sampling proportion is *Normal*
Hypothesis Test  
$H_0$ : $P_{vac} = P_{not vac.}$, $P_{vac} - P_{not vac.} = 0$  
$H_A$ : $P_{vac} \neq P_{not vac.}$

In [57]:
df_vac = df[df['VACCINATED'] == 1]
df_not_vac = df[df['VACCINATED'] != 1]
n_vac = df_vac.shape[0]
n_not_vac = df_not_vac.shape[0]
n_combine = df.shape[0]
p_vac = df_vac['INFECTED'].sum()/n_vac
p_not_vac = df_not_vac['INFECTED'].sum()/n_not_vac
p_combine = df['INFECTED'].sum()/n_combine
se_combine = np.sqrt((p_combine*(1-p_combine))*(1/n_not_vac + 1/n_vac))

In [58]:
print(f'{p_vac}, {n_vac}'); print(f'{p_not_vac}, {n_not_vac}'); print(f'{p_combine}, {n_combine}')

0.005234718000675448, 5922
0.09131293188548864, 6078
0.04883333333333333, 12000


In [65]:
z_test_score = ((p_not_vac - p_vac) - (0))/se_combine

In [66]:
import scipy as sp

print(z_test_score)
p_value = sp.stats.norm.sf(z_test_score)
print(p_value)

21.874124374627076
2.291116351867227e-106


#### 2.2) Sampling distribution test with $Z-test$ , library `statsmodels`
reference : https://sonalake.com/latest/hypothesis-testing-of-proportion-based-samples/

In [67]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

vac_infected, vac_size = (df_vac['INFECTED'].sum(), df_vac.shape[0])
not_vac_infected, not_vac_infected_size = (df_not_vac['INFECTED'].sum(), df_not_vac.shape[0])

# check our sample against Ho for Ha != Ho
total_infected = np.array([not_vac_infected, vac_infected])
total_size = np.array([not_vac_infected_size, vac_size])

# note, no need for a Ho value here - it's derived from the other parameters
stat, p_value = proportions_ztest(count=total_infected, nobs=total_size,  alternative='two-sided')

# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))

z_stat: 21.874, p_value: 0.000


#### 2.3) Bootstraping test  
reference : https://stats.stackexchange.com/questions/395120/proportion-test-z-test-vs-bootstrap-simulation-different-results